# Core package 
In the following set of notebooks, we will go line by line into the bilby library. This would help us understand the tool we're working with at a deep level that would allow us to troubleshoot the problem we are currently facing. In this document we would go through the core package namely the package under the `bilby/core` direcroty, it consists of the following tree:

```
.
├── __init__.py
├── fisher.py
├── grid.py
├── likelihood.py
├── prior
│   ├── __init__.py
│   ├── analytical.py
│   ├── base.py
│   ├── conditional.py
│   ├── dict.py
│   ├── interpolated.py
│   ├── joint.py
│   └── slabspike.py
├── result.py
├── sampler
│   ├── __init__.py
│   ├── base_sampler.py
│   ├── cpnest.py
│   ├── dnest4.py
│   ├── dynamic_dynesty.py
│   ├── dynesty.py
│   ├── dynesty_utils.py
│   ├── emcee.py
│   ├── fake_sampler.py
│   ├── kombine.py
│   ├── nessai.py
│   ├── nestle.py
│   ├── polychord.py
│   ├── proposal.py
│   ├── ptemcee.py
│   ├── ptmcmc.py
│   ├── pymc.py
│   ├── pymultinest.py
│   ├── ultranest.py
│   └── zeus.py
├── series.py
└── utils
    ├── __init__.py
    ├── calculus.py
    ├── cmd.py
    ├── colors.py
    ├── constants.py
    ├── conversion.py
    ├── counter.py
    ├── docs.py
    ├── entry_points.py
    ├── env.py
    ├── introspection.py
    ├── io.py
    ├── log.py
    ├── meta_data.py
    ├── plotting.py
    ├── random.py
    ├── samples.py
    └── series.py
```
We'd go through each file explaining what it does and how it does it. After that we would have a report in the `docs` directory of this repo explaining the important parts, so if you're not into reading the whole package with me just wait and read the docs later on.

# ./Prior/base.py
The first file that we would take a look at is the `base.py` file. It consists of three main classes:

1. `Prior`
2. `Constraint(Prior)`
3. `PriorException(Exception)`

The most important one here is the `Prior` class which weirdly is not in the `./prior.py` file and here instead. Any way let's take a look at the methods it has and have a description of what it does:

## class Prior

This class implements a prior object (TODO: What is a prior object?) the class consists of the following methods:

- `__init__`: Initializer for an instance of Prior class. To make an instance one needs a `name: str` (one associated with prior), a `latex_label:str`, which is a latex string used for plotting. A `unit: str` which is a string, if given it must be a latex string describing the units of the parameter. `minimum:float` is the minimum of the domain (TODO: of what?), by default it is negative infinity. `maximum: float` is the maximum of the domain, and by default it is positive infinity (`np.inf`). `check_range_nonzero: boolean` checks that the prior range is non-zero, it is true by default. `boundary: str` is the boundary condition of prior, can be `periodic` or `reflective` currently implemented in cpnest, dynesty and pymultinest.
```python
class Prior(object):
    # This is a simple variable holding the plotting labels in latex format.
    _default_latex_labels = {}
    # The init function that we've talked about
    def __init__(self, name=None, latex_label=None, unit=None, minimum=-np.inf, maximum=np.inf, check_range_nonzero=True, boundary=None):
        # The function first checks that the range given (minimum and maximum) is non-zero, which means to check if
        # maximum <= minimum
        if check_range_nonzero and maximum <= minimum:
            raise ValueError(
                "maximum {} <= minimum {} for {} prior on {}".format(maximum, minimum, type(self).__name__, name)
            )
        # Otherwise it would initialize the internal values 
        self.name = name
        self.latex_label= latex_label
        self.unit = unit
        self.minimum = minimum
        self.maximum = maximum
        self.check_range_nonzero = check_range_nonzero
        # this ones new and is not initialized using the users input.
        self.least_recently_sampled = None
        self.boundary = boundary
        self._is_fixed= False
```
- `__call__`: Overrides the `__call__` special method. Calls the sample method instead:
```python
def __call__(self)
    return self.sample()
```
- `__eq__`: Test equality for two prior objects. This function overrides the `==` operation between two instances of prior. It returns tue if and only if

1.  The class of the two priors are the same
2.  Both priors have the same keys in the `__dict__` attribute
3.  The instantiation arguments match‍‍‍.
```python
# I think I have to submit a typed version of this: def__eq__(self,other)-> bool instead.
def __eq__(self, other) :
    # The first condition mentioned above:
    if self.__class__ != other.__class__:
        return False # This generally means that if x = SomeRandomClass()  and Y = Prior() we don't need to check internals.
    # The second condition mentioned above:
    # of course we need to sort the class first to not make mistakes:
    if sorted(self.__dict__.keys()) != sorted(self.__dict__.keys()):
        return False
    this_dict = self.get_instantiation_dict()
    other_dict = other.get_instantiation_dict()
    for key in this_dict:
        if == "least_recently_sampled":
            continue
        if isinstance(this_dict[key], np.ndarray):
            if not np.array_equal(this_dict[key], other_dict[key]):
                False
        elif isinstance(this_dict[key], type(scipy.stats.beta(1.,1.))):
            continue
        else:
            if not this_dict[key] == other_dict[key]:
                return False
    return True
```

These methods that we explained matter only internally and are not part of the core functionality of the project. Let's begin investigating the public methods:

- `sample`: This method can be used to draw a sample from the prior. Uses a random number generator to uniformy make a sample. Takes an int or tuple of ints (size). Returns a random number between 0 and 1, rescaled to math the distribution of this Prior.
```python
def sample(self, size=None):
    ## No worries, I haven't read the utils folder but obviously this is just a random number generator:
    from ..utils import random
    self.least_recently_sampled = self.rescale(random.rng.uniform(0,1,size))
    return self.least_recently_sampled
```
- `rescale` and `prob`: Apparently these methods were to be overrided by the subclasses of the project and are technically empty in the core Prior class. We would just keep in mind that such things exist.
- `cdf`: This one as well can be overrided in subclasses. But we have a generic implementation of it here as well:
```python
def cdf(self, val):
    # imports integration method of scipy named cumulative trapezoid 
    # you can read about this method of integrating in wikipedia 
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html
    # https://www.wikiwand.com/en/articles/Trapezoidal_rule
    from scipy.integrate import cumulative_trapezoid
    # So we get a hint here that although minimum and maximum are set to be infinite at the initialization
    # by the default values, they must be initialized after all :O...
    if np.any(np.isinf([self.minimum, self.maximum]):
        raise ValueError("...")

    # First it makes a linear sample
    # Weird we have a constant 1000 number here... Wonder if it can be set by the user in any way.
    x = np.linspace(self.minimum, self.maximum, 1000)
    pdf = self.prob(x) # Using a function that doesn't exist yet.
    # integrating over the pair <pdf,x> 
    cdf = cumulative_trapezoid(pdf, x, initial=0)
    # Interpolation, this function helps us use samples (discrete data) as something smooth (continuous) by
    # guessing a number between the present numbers.
    interp = interp1d(x, cdf, assume_sorted=True, bounds_error=False, fill_value=(0,1))

    # we return the ineterpolated value of the function
    return interp(val)
```
- `ln_prob`: Returns the natural log of prob given the value.
- `is_in_prior_range` checks if the value is in the prior range: `val >= self.minimum & val <= self.maximum`
- `__repr__`: Returns a representation of this instance that resembles how it is instantiated.

The class declration proceeds with getters and setters for different values, the most important one is the `@boundary.setter` annotation above the `def boundary(self, boundary)` function which sets prior boundaries.

### Class methods
- `from_joson`: reads a json file and makes a prior instance
- `from_repr`:  I assume the same thing happens as `from_json`

## class Constraint
This is the first subclass of Prior defines as below:
```python
class Constraint(Prior):
    # nothing interesting happens here except the is fixed variable which is set to true.
    def __init__(self, minimum, maximum, name=None, latex_label=None, unit=None):
        super(Constrains, self).__init__(minimum=minimum, maximum=maximum, name=name, latex_label=latex_label, unit=unit)
        self._is_fixed = True
    
    # as you remember prob wasn't implemented int he main Prior class, so here's the override of the method.
    def prob(self, val):
        return (val > self.minimum) & ( val < self.maximum)
```
        

## class PriorDict
This class is a subtype of internal dict class of python. This holds a dictionary of priors. 
- `__init__`: For creating an instance of the PriorDict there are three values, two of which are declaration of data `dictionary`, if given, a dictionary to generate prior set. `filename` is a file containing the prior to generate the prior set. `conversion_function` is a function to convert between sampled parameters and constraints default is not conversion.
```python
class PriorDict(dict):
    def __init__(self, dictionary=None, filename=None, conversion_function=None):
        super(PriorDict, self).__init__()
        # Simply checking different inputs and transforming them into PriorDict object.
        # First if we gave a dictionary, it simply uses from dictionary function to load it.
        if isinstance(dictionary, dict):
            self.from_dictionary(dictionary)
        # If the dictionary field was filled witha string, the class assumes its a file path.
        elif type(dictionary) is str:
            logger.debug("...")
            self.from_file(dictionary)
        # If the user actually filled the filename field.
        elif type(filename) is str:
            self.from_file(filename)
        # other than these three case we just have errors
        elif dictionary is not None:
            raise ValueError("...")

        self._cached_normalizations = {} # ???
        self.convert_floats_to_delta_functions() # ???

        # Setting the conversion_function and a default function if the user didn't provide.
        if conversion_function is not None:
            self.conversion_function = conversion_function
        else:
            self.conversion_function = self.default_conversion_function
```
- `evaluate_constraints`: Din't 